In [16]:
import sys 
import os 
#sys.path.append('../../')
sys.path.append('../')

import numpy as np
from numpy import concatenate
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM,Dropout
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.cluster import KMeans
import warnings
import os
warnings.filterwarnings("ignore")

from utils import utilidades as myutils

In [17]:
def clean_group_data(dataset, parameter, range_upper = 1,range_lower = 1):
    
    q1, q3 = np.percentile(dataset[parameter],[25,75])
    iqr = q3 - q1
    lower_bound = q3 - range_upper*(1.5 * iqr)  
    upper_bound = q3 + range_upper*(1.5 * iqr)  

    # Deleting lower bound and upper bound from the dataset LinkTT2
    #dataset = dataset.loc[(dataset[parameter] >= lower_bound) & 
                                              #(dataset[parameter] <= upper_bound)]
    #dataset.interpolate()
    dataset.interpolate(method = 'pad')
    
    return dataset

def resultado(df,tecnica,parameter,amostras,metrica,lag):
    
    df.loc[len(df)]= [tecnica,parameter,amostras,round(metrica,2),lag]
    
    
    return df



In [18]:
parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    DFmerged = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  

    

    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    DFmerged = DFmerged[DFmerged['codigo_ponto'] == 'TIET02050']    
    DFmerged = clean_group_data(DFmerged, 'valor')
    
    DFmerged['data_coleta'] = pd.to_datetime(DFmerged['data_coleta'])
    DFmerged.index = DFmerged['data_coleta']    
    DFmerged = DFmerged.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged = DFmerged.loc['1979-01-31':]
    
    
    scaler = MinMaxScaler()
    #DFmerged = scaler.fit_transform(np.log(DFmerged + 0.0000001))
#     DFmerged = scaler.fit_transform()

    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
    for lag in range(1,10):

        X  = DFmerged[:-lag]
        y  = DFmerged[lag:]


        dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
        dataset.columns = ['X','y']
        dataset.dropna(inplace=True)

        train_X = scaler.fit_transform(np.array(dataset['X']).reshape(len(dataset), 1))
        train_y = scaler.fit_transform(np.log(np.array(dataset['y']).reshape(len(dataset), 1) + 0.000000001))
        
        #tamanho_treino = int(len(train_X)*0.7)
        tamanho_teste  = int(len(train_y)*0.3)

        #Desnormalização train_y(teste)
        y_teste = scaler.inverse_transform(train_y[-tamanho_teste:])

        #Regressão Linear
        model = LinearRegression(normalize=False)

        #model.fit(train_X[:-40],train_y[:-40])
        model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
        score = model.score(train_X,train_y)
    #     print(score)

        #Dados de teste
        #previsoes = model.predict(train_X[-40:])
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)

        #df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),myutils.mean_absolute_percentage_error(train_y[-40:], previsoes))
        df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),lag)
        
        
        #print(parameter, len(DFmerged),'MAPE',str(myutils.mean_absolute_percentage_error(train_y[-40:], previsoes)))

        #Random Forest
        model = RandomForestRegressor()

        model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
        score = model.score(train_X,train_y)
    #     print(score)

        #Dados de teste
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes.reshape(-1, 1))

        df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),lag)

        #MLP
        model = Sequential()
        model.add(Dense(units = 10, activation = 'relu', input_dim = train_X[:-tamanho_teste].shape[1]))
        model.add(Dense(units = 21, activation = 'relu'))
        model.add(Dense(units = 1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste], validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(train_X[-tamanho_teste:])
            previsoes = scaler.inverse_transform(previsoes)

            media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))  

        df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged),np.mean(media_previsoes),lag)


        #LSTM

        train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
        train_y = np.reshape(train_y, (train_y.shape[0], train_y.shape[1], 1))


        model = Sequential()
        model.add(LSTM(units = 10, input_shape = (train_X[:-tamanho_teste].shape[1], 1)))
        model.add(Dense(21, activation = 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste],validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(train_X[-tamanho_teste:])
            previsoes = scaler.inverse_transform(previsoes)
            media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))

        df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged),np.mean(media_previsoes),lag)

        '''
        g_X = train_X.reshape((train_X.shape[0],train_X.shape[1]))
        g_previsoes = concatenate((previsoes,g_X[-tamanho_teste:]),axis=1)
        g_previsoes = scaler.inverse_transform(g_previsoes)
        #g_previsoes = g_previsoes[:,1]

        g_y   = train_y.reshape(len(train_y),1)
        g_obs = concatenate((g_y[-tamanho_teste:],g_X[-tamanho_teste:]),axis=1)
        g_obs = scaler.inverse_transform(g_obs)
       # g_obs = g_obs[:,1]

        #plt.clf()
        plt.plot(g_previsoes,label="model")
        plt.plot(g_obs,label="Observed")
        plt.legend()
        plt.title(parameter)
        plt.savefig(parameter+".png")
        plt.close("all")
        ''' 
    
df_result

,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,170,33.37,1
1,Random Forest,Coliformes Termotolerantes,170,51.95,1
2,MLP,Coliformes Termotolerantes,170,34.32,1
3,LSTM,Coliformes Termotolerantes,170,33.18,1
4,Regressão Linear,Coliformes Termotolerantes,170,33.37,2
...,...,...,...,...,...
283,LSTM,pH,207,1.87,8
284,Regressão Linear,pH,207,1.83,9
285,Random Forest,pH,207,2.48,9
286,MLP,pH,207,1.83,9


In [19]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/temporal_interpolacao_pad_lag.csv', index = True, header=True)

In [20]:
#predição espaço-temporal
print('predição espaço-temporal')

parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    
    DFmerged1 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  
    
    DFmerged2 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02090') &
                      (DFmerge['parametro'] == parameter)]

    
    
    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    DFmerged1 = DFmerged1[DFmerged1['codigo_ponto'] == 'TIET02050']    
    DFmerged1 = clean_group_data(DFmerged1, 'valor')
    
    DFmerged2 = DFmerged2[DFmerged2['codigo_ponto'] == 'TIET02090']    
    DFmerged2 = clean_group_data(DFmerged2, 'valor')
    
     
    
    DFmerged1['data_coleta'] = pd.to_datetime(DFmerged1['data_coleta'])
    DFmerged1.index = DFmerged1['data_coleta']    
    DFmerged1 = DFmerged1.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged1 = DFmerged1.loc['1979-01-31':]
       
    DFmerged2['data_coleta'] = pd.to_datetime(DFmerged2['data_coleta'])
    DFmerged2.index = DFmerged2['data_coleta']    
    DFmerged2 = DFmerged2.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged2 = DFmerged2.loc['1979-01-31':]
    
    #coloca o dataframes com o mesmo tamanho
    #DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    if (len(DFmerged1) > len(DFmerged2)):
        DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    else:
        DFmerged2 = DFmerged2.iloc[0:len(DFmerged1)]   
    
    
    scaler = MinMaxScaler()
    #DFmerged = scaler.fit_transform(np.log(DFmerged + 0.0000001))
#     DFmerged = scaler.fit_transform()

    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
    for lag in range(1,10):

        X  = DFmerged1[:-lag]
        y  = DFmerged2[lag:]


        dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
        dataset.columns = ['X','y']
        dataset.dropna(inplace=True)

        train_X = scaler.fit_transform(np.array(dataset['X']).reshape(len(dataset), 1))
        train_y = scaler.fit_transform(np.log(np.array(dataset['y']).reshape(len(dataset), 1) + 0.000000001))
        
        #tamanho_treino = int(len(train_X)*0.7)
        tamanho_teste  = int(len(train_y)*0.3)

        #Desnormalização train_y(teste)
        y_teste = scaler.inverse_transform(train_y[-tamanho_teste:])

        #Regressão Linear
        model = LinearRegression(normalize=False)

        #model.fit(train_X[:-40],train_y[:-40])
        model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
        score = model.score(train_X,train_y)
    #     print(score)

        #Dados de teste
        #previsoes = model.predict(train_X[-40:])
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)

        #df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),myutils.mean_absolute_percentage_error(train_y[-40:], previsoes))
        df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged1),myutils.mean_absolute_percentage_error(y_teste, previsoes),lag)
        
        
        #print(parameter, len(DFmerged),'MAPE',str(myutils.mean_absolute_percentage_error(train_y[-40:], previsoes)))

        #Random Forest
        model = RandomForestRegressor()

        model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
        score = model.score(train_X,train_y)
    #     print(score)

        #Dados de teste
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes.reshape(-1, 1))

        df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged1),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),lag)

        #MLP
        model = Sequential()
        model.add(Dense(units = 10, activation = 'relu', input_dim = train_X[:-tamanho_teste].shape[1]))
        model.add(Dense(units = 21, activation = 'relu'))
        model.add(Dense(units = 1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste], validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(train_X[-tamanho_teste:])
            previsoes = scaler.inverse_transform(previsoes)

            media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))  

        df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged1),np.mean(media_previsoes),lag)


        #LSTM

        train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
        train_y = np.reshape(train_y, (train_y.shape[0], train_y.shape[1], 1))


        model = Sequential()
        model.add(LSTM(units = 10, input_shape = (train_X[:-tamanho_teste].shape[1], 1)))
        model.add(Dense(21, activation = 'relu'))
        model.add(Dropout(0.2))
        model.add(Dense(1, activation = 'sigmoid'))
        model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

        es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

        #Treina o modelo
        history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste],validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

        #Dados de teste
        media_previsoes = []
        for r in range(0,6):
            previsoes = model.predict(train_X[-tamanho_teste:])
            previsoes = scaler.inverse_transform(previsoes)
            media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))

        df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged1),np.mean(media_previsoes),lag)

         
    
df_result

predição espaço-temporal


,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,166,29.69,1
1,Random Forest,Coliformes Termotolerantes,166,34.98,1
2,MLP,Coliformes Termotolerantes,166,33.65,1
3,LSTM,Coliformes Termotolerantes,166,32.90,1
4,Regressão Linear,Coliformes Termotolerantes,166,29.43,2
...,...,...,...,...,...
283,LSTM,pH,202,1.34,8
284,Regressão Linear,pH,202,1.34,9
285,Random Forest,pH,202,1.79,9
286,MLP,pH,202,1.34,9


In [21]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/espaco_temporal_interpolacao_pad_lag.csv', index = True, header=True)

In [22]:
#predição espacial
print('predição espacial')

parameters = ['coliformes', 'od', 'turbidez', 'fosforo', 'solido', 'dbo', 'temperatura', 'ph']
df_resultado = pd.DataFrame(columns=['tecnica','parametro','amostras','mape','lag'])

DFmerge = pd.DataFrame()
for parameter in parameters:
    # Concatenating all the parameter files
    df = pd.read_csv('../data/CETESB/'+ parameter + '.csv',encoding='utf-8',sep=';')
    DFmerge = pd.concat([DFmerge, df])
    
    
for parameter in DFmerge['parametro'].unique():    
    
    DFmerged1 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02050') &
                      (DFmerge['parametro'] == parameter)]  
    
    DFmerged2 = DFmerge[(DFmerge['codigo_ponto'] == 'TIET02090') &
                      (DFmerge['parametro'] == parameter)]

    
    
    # DFmerge.groupby([pd.Grouper(freq='1M'), 'codigo_ponto']).mean().unstack()
    # DFmerge.groupby(['codigo_ponto', 'UGRHI'])['valor'].count().unstack()

    DFmerged1 = DFmerged1[DFmerged1['codigo_ponto'] == 'TIET02050']    
    DFmerged1 = clean_group_data(DFmerged1, 'valor')
    
    DFmerged2 = DFmerged2[DFmerged2['codigo_ponto'] == 'TIET02090']    
    DFmerged2 = clean_group_data(DFmerged2, 'valor')
    
     
    
    DFmerged1['data_coleta'] = pd.to_datetime(DFmerged1['data_coleta'])
    DFmerged1.index = DFmerged1['data_coleta']    
    DFmerged1 = DFmerged1.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged1 = DFmerged1.loc['1979-01-31':]
       
    DFmerged2['data_coleta'] = pd.to_datetime(DFmerged2['data_coleta'])
    DFmerged2.index = DFmerged2['data_coleta']    
    DFmerged2 = DFmerged2.groupby([pd.Grouper(freq='1M'), 'parametro'])['valor'].mean().unstack()
    DFmerged2 = DFmerged2.loc['1979-01-31':]
    
    #coloca o dataframes com o mesmo tamanho
    #DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    if (len(DFmerged1) > len(DFmerged2)):
        DFmerged1 = DFmerged1.iloc[0:len(DFmerged2)]
    else:
        DFmerged2 = DFmerged2.iloc[0:len(DFmerged1)]   
    
    
    scaler = MinMaxScaler()
    #DFmerged = scaler.fit_transform(np.log(DFmerged + 0.0000001))
#     DFmerged = scaler.fit_transform()

    # #Mostra a quantidade de NaN no dataframe
    #for field in DFmerge.columns:
        #print(field, 'NaN:', DFmerge[field].isnull().sum())
    #print(parameter)    
    #print('Valor', 'NaN:', DFmerge['valor'].isnull().sum())
    
        
    X  = DFmerged1[:]
    y  = DFmerged2[:]


    dataset = pd.DataFrame(np.concatenate([X,y], axis=1))
    dataset.columns = ['X','y']
    dataset.dropna(inplace=True)

    train_X = scaler.fit_transform(np.array(dataset['X']).reshape(len(dataset), 1))
    train_y = scaler.fit_transform(np.log(np.array(dataset['y']).reshape(len(dataset), 1) + 0.000000001))
        
    #tamanho_treino = int(len(train_X)*0.7)
    tamanho_teste  = int(len(train_y)*0.3)

    #Desnormalização train_y(teste)
    y_teste = scaler.inverse_transform(train_y[-tamanho_teste:])

    #Regressão Linear
    model = LinearRegression(normalize=False)

    #model.fit(train_X[:-40],train_y[:-40])
    model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
    score = model.score(train_X,train_y)
    #     print(score)

    #Dados de teste
    #previsoes = model.predict(train_X[-40:])
    previsoes = model.predict(train_X[-tamanho_teste:])
    previsoes = scaler.inverse_transform(previsoes)

    #df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged),myutils.mean_absolute_percentage_error(train_y[-40:], previsoes))
    df_result = resultado(df_resultado,'Regressão Linear',parameter,len(DFmerged1),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),'')
        
        
    #print(parameter, len(DFmerged),'MAPE',str(myutils.mean_absolute_percentage_error(train_y[-40:], previsoes)))

    #Random Forest
    model = RandomForestRegressor()

    model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste])
    score = model.score(train_X,train_y)
    #     print(score)

    #Dados de teste
    previsoes = model.predict(train_X[-tamanho_teste:])
    previsoes = scaler.inverse_transform(previsoes.reshape(-1, 1))

    df_result = resultado(df_resultado,'Random Forest',parameter,len(DFmerged1),
                              myutils.mean_absolute_percentage_error(y_teste, previsoes),'')

    #MLP
    model = Sequential()
    model.add(Dense(units = 10, activation = 'relu', input_dim = train_X[:-tamanho_teste].shape[1]))
    model.add(Dense(units = 21, activation = 'relu'))
    model.add(Dense(units = 1, activation = 'sigmoid'))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

    es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

    #Treina o modelo
    history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste], validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]),  batch_size = 32, epochs = 2000, callbacks=[es], verbose=0)

    #Dados de teste
    media_previsoes = []
    for r in range(0,6):
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)

        media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))  

    df_result = resultado(df_resultado,'MLP',parameter,len(DFmerged1),np.mean(media_previsoes),'')


    #LSTM

    train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))
    train_y = np.reshape(train_y, (train_y.shape[0], train_y.shape[1], 1))


    model = Sequential()
    model.add(LSTM(units = 10, input_shape = (train_X[:-tamanho_teste].shape[1], 1)))
    model.add(Dense(21, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'mean_absolute_error', optimizer = 'adam',metrics = ['mean_absolute_error'])

    es = EarlyStopping(monitor='val_loss', patience = 5, verbose=0)

    #Treina o modelo
    history = model.fit(train_X[:-tamanho_teste],train_y[:-tamanho_teste],validation_data = (train_X[:-tamanho_teste],train_y[:-tamanho_teste]), batch_size = 32, epochs = 100, callbacks=[es], verbose=0)

    #Dados de teste
    media_previsoes = []
    for r in range(0,6):
        previsoes = model.predict(train_X[-tamanho_teste:])
        previsoes = scaler.inverse_transform(previsoes)
        media_previsoes.append(myutils.mean_absolute_percentage_error(y_teste, previsoes))

    df_result = resultado(df_resultado,'LSTM',parameter,len(DFmerged1),np.mean(media_previsoes),'')

         
    
df_result

predição espacial


,tecnica,parametro,amostras,mape,lag
0,Regressão Linear,Coliformes Termotolerantes,166,29.36,
1,Random Forest,Coliformes Termotolerantes,166,34.43,
2,MLP,Coliformes Termotolerantes,166,33.23,
3,LSTM,Coliformes Termotolerantes,166,32.98,
4,Regressão Linear,Oxigênio Dissolvido,201,32.99,
5,Random Forest,Oxigênio Dissolvido,201,47.58,
6,MLP,Oxigênio Dissolvido,201,42.54,
7,LSTM,Oxigênio Dissolvido,201,51.43,
8,Regressão Linear,Turbidez,200,46.14,
9,Random Forest,Turbidez,200,48.82,


In [23]:
df_result.to_csv (r'/home/anderson/Downloads/predicaoagua/src/espacial_interpolacao_pad.csv', index = True, header=True)